## Imports

In [1]:
# pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
        # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


In [2]:
!pwd


/home/aimwps/PyCode/SAI/SAImwps/M4_Feature_Eng/01. Robust ML


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"



## Load data

In [4]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

df.head()

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Check missings

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [7]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda x: x.split(",")[1].split(".")[0].strip()

# YOUR CODE HERE
df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

print(df.Title.unique())
print(df_test.Title.unique())

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer']
['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


In [8]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [9]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [10]:
# Use map to apply the prevous dict

df["Title"] =  df['Title'].map(title_dictionary)
print(df.Title.unique())
df_test["Title"] = df_test['Title'].map(title_dictionary, na_action='ignore')
print(df_test.Title.unique())
# YOUR CODE HERE

['Mr' 'Mrs' 'Miss' 'Master' 'Royalty' 'Officer']
['Mr' 'Mrs' 'Miss' 'Master' 'Officer' 'Royalty']


In [11]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [12]:
print(len(df['Ticket']))
len(df['Ticket'].unique())



891


681

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [13]:
get_deck_from_cabin = lambda x: x[0] if isinstance(x, str) else "Z"  
df['Cabin'] = df['Cabin'].map(get_deck_from_cabin)
df_test['Cabin'] = df_test['Cabin'].map(get_deck_from_cabin)
print(df.head())

             Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin Embarked Title  
PassengerId                                                                
1                1      0         A/5 21171   7.2500   

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [14]:
x = df.drop(columns=["Survived", 'Name', 'Ticket']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket']) # # X_TEST DATA (NEW DATA)

In [15]:
cat_vars  = ['Sex', 'Embarked', 'Title','Cabin']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title', 'Cabin']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [16]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='median', add_indicator=False))]) # mean, median

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan))])
tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),],
    remainder='drop')
                                     
tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=nan))]),
                                 ['Sex', 'Embarked', 'Title', 'Cabin'])])

In [17]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [18]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [19]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost":XGBClassifier(use_label_encoder=False),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers['LightGBM']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan))]),
                                                  ['Sex', 'Embarked', 'Title',
                                                   'Cabin'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [20]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [21]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size = 0.2,
    stratify=y,
    random_state=909)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    model.fit(x_train, y_train)
    # GET PREDICTIONS USING x_val
    pred = model.predict(x_val)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
#                               "Accuracy": metrics.accuracy_score(y_val, pred)*100,
#                               "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

[16:27:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6872602	total: 53.3ms	remaining: 53.2s
1:	learn: 0.6814074	total: 58.9ms	remaining: 29.4s
2:	learn: 0.6754831	total: 65.3ms	remaining: 21.7s
3:	learn: 0.6707478	total: 70.2ms	remaining: 17.5s
4:	learn: 0.6649667	total: 75.8ms	remaining: 15.1s
5:	learn: 0.6604238	total: 77.6ms	remaining: 12.9s
6:	learn: 0.6553019	total: 79.5ms	remaining: 11.3s
7:	learn: 0.6510688	total: 81.6ms	remaining: 10.1s
8:	learn: 0.6456872	total: 85.8ms	remaining: 9.44s
9:	learn: 0.6406840	total: 88.3ms	remaining: 8.75s
10:	learn: 0.6362158	total: 92.7ms	remaining: 8.33s
11:	learn: 0.6314003	total: 97ms	remaining: 7.98s
12:	learn: 0.6272956	total: 102ms	remaining: 7.74s
13:	learn: 0.6227810	total: 107ms	remaining: 7.53s
14:	

162:	learn: 0.3987127	total: 367ms	remaining: 1.88s
163:	learn: 0.3986099	total: 368ms	remaining: 1.88s
164:	learn: 0.3982362	total: 374ms	remaining: 1.89s
165:	learn: 0.3976019	total: 375ms	remaining: 1.88s
166:	learn: 0.3973171	total: 376ms	remaining: 1.88s
167:	learn: 0.3967835	total: 377ms	remaining: 1.87s
168:	learn: 0.3965027	total: 378ms	remaining: 1.86s
169:	learn: 0.3960777	total: 381ms	remaining: 1.86s
170:	learn: 0.3955372	total: 384ms	remaining: 1.86s
171:	learn: 0.3951931	total: 386ms	remaining: 1.86s
172:	learn: 0.3947047	total: 387ms	remaining: 1.85s
173:	learn: 0.3943239	total: 390ms	remaining: 1.85s
174:	learn: 0.3938337	total: 394ms	remaining: 1.86s
175:	learn: 0.3934633	total: 397ms	remaining: 1.86s
176:	learn: 0.3931131	total: 400ms	remaining: 1.86s
177:	learn: 0.3928589	total: 403ms	remaining: 1.86s
178:	learn: 0.3924431	total: 405ms	remaining: 1.85s
179:	learn: 0.3919436	total: 406ms	remaining: 1.85s
180:	learn: 0.3914048	total: 409ms	remaining: 1.85s
181:	learn: 

337:	learn: 0.3527259	total: 727ms	remaining: 1.42s
338:	learn: 0.3525878	total: 731ms	remaining: 1.43s
339:	learn: 0.3525353	total: 733ms	remaining: 1.42s
340:	learn: 0.3523000	total: 734ms	remaining: 1.42s
341:	learn: 0.3521256	total: 736ms	remaining: 1.42s
342:	learn: 0.3519959	total: 737ms	remaining: 1.41s
343:	learn: 0.3518629	total: 739ms	remaining: 1.41s
344:	learn: 0.3516988	total: 740ms	remaining: 1.4s
345:	learn: 0.3514423	total: 744ms	remaining: 1.41s
346:	learn: 0.3512366	total: 745ms	remaining: 1.4s
347:	learn: 0.3510589	total: 748ms	remaining: 1.4s
348:	learn: 0.3508875	total: 750ms	remaining: 1.4s
349:	learn: 0.3506815	total: 751ms	remaining: 1.39s
350:	learn: 0.3505968	total: 752ms	remaining: 1.39s
351:	learn: 0.3504847	total: 755ms	remaining: 1.39s
352:	learn: 0.3502705	total: 756ms	remaining: 1.39s
353:	learn: 0.3500759	total: 757ms	remaining: 1.38s
354:	learn: 0.3498785	total: 759ms	remaining: 1.38s
355:	learn: 0.3497107	total: 760ms	remaining: 1.37s
356:	learn: 0.34

570:	learn: 0.3201243	total: 1.08s	remaining: 814ms
571:	learn: 0.3200358	total: 1.08s	remaining: 812ms
572:	learn: 0.3199438	total: 1.08s	remaining: 809ms
573:	learn: 0.3198447	total: 1.09s	remaining: 807ms
574:	learn: 0.3195994	total: 1.09s	remaining: 805ms
575:	learn: 0.3195104	total: 1.09s	remaining: 802ms
576:	learn: 0.3193313	total: 1.09s	remaining: 800ms
577:	learn: 0.3191450	total: 1.09s	remaining: 798ms
578:	learn: 0.3189994	total: 1.09s	remaining: 796ms
579:	learn: 0.3187489	total: 1.09s	remaining: 793ms
580:	learn: 0.3187369	total: 1.09s	remaining: 790ms
581:	learn: 0.3184776	total: 1.1s	remaining: 788ms
582:	learn: 0.3183348	total: 1.1s	remaining: 785ms
583:	learn: 0.3182710	total: 1.1s	remaining: 783ms
584:	learn: 0.3178416	total: 1.1s	remaining: 780ms
585:	learn: 0.3177212	total: 1.1s	remaining: 778ms
586:	learn: 0.3175907	total: 1.1s	remaining: 775ms
587:	learn: 0.3174444	total: 1.1s	remaining: 773ms
588:	learn: 0.3173757	total: 1.1s	remaining: 771ms
589:	learn: 0.317203

802:	learn: 0.2904659	total: 1.44s	remaining: 354ms
803:	learn: 0.2902263	total: 1.44s	remaining: 352ms
804:	learn: 0.2901417	total: 1.44s	remaining: 350ms
805:	learn: 0.2900715	total: 1.45s	remaining: 348ms
806:	learn: 0.2899795	total: 1.45s	remaining: 347ms
807:	learn: 0.2899125	total: 1.45s	remaining: 345ms
808:	learn: 0.2898029	total: 1.45s	remaining: 343ms
809:	learn: 0.2897155	total: 1.45s	remaining: 341ms
810:	learn: 0.2895807	total: 1.46s	remaining: 340ms
811:	learn: 0.2894118	total: 1.46s	remaining: 338ms
812:	learn: 0.2892929	total: 1.46s	remaining: 336ms
813:	learn: 0.2891369	total: 1.46s	remaining: 334ms
814:	learn: 0.2889041	total: 1.47s	remaining: 333ms
815:	learn: 0.2887900	total: 1.47s	remaining: 331ms
816:	learn: 0.2885192	total: 1.47s	remaining: 330ms
817:	learn: 0.2883544	total: 1.47s	remaining: 328ms
818:	learn: 0.2881547	total: 1.48s	remaining: 326ms
819:	learn: 0.2879786	total: 1.48s	remaining: 324ms
820:	learn: 0.2878455	total: 1.48s	remaining: 323ms
821:	learn: 

988:	learn: 0.2649239	total: 1.81s	remaining: 20.1ms
989:	learn: 0.2648220	total: 1.81s	remaining: 18.3ms
990:	learn: 0.2647694	total: 1.81s	remaining: 16.5ms
991:	learn: 0.2647007	total: 1.81s	remaining: 14.6ms
992:	learn: 0.2646454	total: 1.81s	remaining: 12.8ms
993:	learn: 0.2645334	total: 1.82s	remaining: 11ms
994:	learn: 0.2643117	total: 1.82s	remaining: 9.14ms
995:	learn: 0.2642555	total: 1.82s	remaining: 7.31ms
996:	learn: 0.2641099	total: 1.82s	remaining: 5.48ms
997:	learn: 0.2640574	total: 1.82s	remaining: 3.65ms
998:	learn: 0.2639380	total: 1.82s	remaining: 1.83ms
999:	learn: 0.2637521	total: 1.82s	remaining: 0us


,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,84.916201,83.675889,0.164258
2,LightGBM,84.357542,82.951252,0.256890
3,XGBoost,83.798883,81.956522,0.459254
4,CatBoost,83.798883,82.226614,2.144796
5,Random Forest,83.240223,82.312253,0.262260
6,Skl HistGBM,83.240223,81.501976,1.601163
7,AdaBoost,82.122905,81.943347,0.135866
8,Extra Trees,81.564246,80.138340,0.231390
9,Decision Tree,77.653631,76.146245,0.040613


In [22]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [35]:
"""
skf = model_selection.StratifiedKFold(
    # CODE HERE

"""
# YOUR CODE HERE
skf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

print(x.shape)
print(y.shape)
for model_name, model in tree_classifiers.items():
    start_time = time.time()

    pred = model_selection.cross_val_predict(model, x, y,cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)

                            
# YOUR CODE HERE


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

(891, 8)
(891,)
[16:20:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:53] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:53] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:54] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to re

165:	learn: 0.3944271	total: 567ms	remaining: 2.85s
166:	learn: 0.3938831	total: 569ms	remaining: 2.84s
167:	learn: 0.3935444	total: 576ms	remaining: 2.85s
168:	learn: 0.3932885	total: 577ms	remaining: 2.84s
169:	learn: 0.3927527	total: 579ms	remaining: 2.82s
170:	learn: 0.3924125	total: 581ms	remaining: 2.81s
171:	learn: 0.3920099	total: 582ms	remaining: 2.8s
172:	learn: 0.3915756	total: 584ms	remaining: 2.79s
173:	learn: 0.3911815	total: 589ms	remaining: 2.8s
174:	learn: 0.3908376	total: 590ms	remaining: 2.78s
175:	learn: 0.3903054	total: 592ms	remaining: 2.77s
176:	learn: 0.3899395	total: 594ms	remaining: 2.76s
177:	learn: 0.3895043	total: 597ms	remaining: 2.76s
178:	learn: 0.3891695	total: 600ms	remaining: 2.75s
179:	learn: 0.3887225	total: 603ms	remaining: 2.75s
180:	learn: 0.3885123	total: 605ms	remaining: 2.74s
181:	learn: 0.3880859	total: 608ms	remaining: 2.73s
182:	learn: 0.3876434	total: 610ms	remaining: 2.72s
183:	learn: 0.3872753	total: 613ms	remaining: 2.72s
184:	learn: 0.

324:	learn: 0.3537926	total: 1.14s	remaining: 2.36s
325:	learn: 0.3535053	total: 1.14s	remaining: 2.36s
326:	learn: 0.3534599	total: 1.14s	remaining: 2.35s
327:	learn: 0.3533431	total: 1.14s	remaining: 2.34s
328:	learn: 0.3532267	total: 1.14s	remaining: 2.33s
329:	learn: 0.3531610	total: 1.15s	remaining: 2.33s
330:	learn: 0.3530910	total: 1.15s	remaining: 2.33s
331:	learn: 0.3529808	total: 1.16s	remaining: 2.33s
332:	learn: 0.3528052	total: 1.16s	remaining: 2.33s
333:	learn: 0.3528034	total: 1.16s	remaining: 2.32s
334:	learn: 0.3526212	total: 1.17s	remaining: 2.32s
335:	learn: 0.3525466	total: 1.17s	remaining: 2.32s
336:	learn: 0.3523648	total: 1.18s	remaining: 2.31s
337:	learn: 0.3521028	total: 1.18s	remaining: 2.31s
338:	learn: 0.3518879	total: 1.19s	remaining: 2.31s
339:	learn: 0.3517414	total: 1.19s	remaining: 2.31s
340:	learn: 0.3516578	total: 1.19s	remaining: 2.3s
341:	learn: 0.3514989	total: 1.19s	remaining: 2.29s
342:	learn: 0.3513335	total: 1.19s	remaining: 2.29s
343:	learn: 0

535:	learn: 0.3259464	total: 1.5s	remaining: 1.3s
536:	learn: 0.3257106	total: 1.51s	remaining: 1.3s
537:	learn: 0.3255924	total: 1.51s	remaining: 1.29s
538:	learn: 0.3255030	total: 1.51s	remaining: 1.29s
539:	learn: 0.3254876	total: 1.51s	remaining: 1.29s
540:	learn: 0.3253425	total: 1.51s	remaining: 1.28s
541:	learn: 0.3252716	total: 1.51s	remaining: 1.28s
542:	learn: 0.3252273	total: 1.52s	remaining: 1.28s
543:	learn: 0.3250429	total: 1.52s	remaining: 1.27s
544:	learn: 0.3250211	total: 1.52s	remaining: 1.27s
545:	learn: 0.3248585	total: 1.52s	remaining: 1.26s
546:	learn: 0.3246008	total: 1.52s	remaining: 1.26s
547:	learn: 0.3244393	total: 1.52s	remaining: 1.25s
548:	learn: 0.3241367	total: 1.53s	remaining: 1.25s
549:	learn: 0.3240999	total: 1.53s	remaining: 1.25s
550:	learn: 0.3240196	total: 1.53s	remaining: 1.25s
551:	learn: 0.3239091	total: 1.53s	remaining: 1.24s
552:	learn: 0.3237987	total: 1.53s	remaining: 1.24s
553:	learn: 0.3235997	total: 1.53s	remaining: 1.24s
554:	learn: 0.3

756:	learn: 0.3023065	total: 1.87s	remaining: 600ms
757:	learn: 0.3022464	total: 1.87s	remaining: 597ms
758:	learn: 0.3021652	total: 1.87s	remaining: 594ms
759:	learn: 0.3021487	total: 1.87s	remaining: 591ms
760:	learn: 0.3019851	total: 1.87s	remaining: 589ms
761:	learn: 0.3019252	total: 1.88s	remaining: 586ms
762:	learn: 0.3018299	total: 1.88s	remaining: 583ms
763:	learn: 0.3017370	total: 1.88s	remaining: 580ms
764:	learn: 0.3016685	total: 1.88s	remaining: 578ms
765:	learn: 0.3014469	total: 1.88s	remaining: 575ms
766:	learn: 0.3012550	total: 1.88s	remaining: 572ms
767:	learn: 0.3010944	total: 1.88s	remaining: 569ms
768:	learn: 0.3010503	total: 1.89s	remaining: 567ms
769:	learn: 0.3007390	total: 1.89s	remaining: 564ms
770:	learn: 0.3005375	total: 1.89s	remaining: 562ms
771:	learn: 0.3005090	total: 1.89s	remaining: 559ms
772:	learn: 0.3003885	total: 1.9s	remaining: 557ms
773:	learn: 0.3003499	total: 1.9s	remaining: 555ms
774:	learn: 0.3002970	total: 1.9s	remaining: 553ms
775:	learn: 0.3

...

144:	learn: 0.3973144	total: 177ms	remaining: 1.04s
145:	learn: 0.3969006	total: 179ms	remaining: 1.05s
146:	learn: 0.3964744	total: 180ms	remaining: 1.04s
147:	learn: 0.3959770	total: 181ms	remaining: 1.04s
148:	learn: 0.3953062	total: 183ms	remaining: 1.04s
149:	learn: 0.3950032	total: 184ms	remaining: 1.04s
150:	learn: 0.3947202	total: 185ms	remaining: 1.04s
151:	learn: 0.3941049	total: 187ms	remaining: 1.04s
152:	learn: 0.3934346	total: 188ms	remaining: 1.04s
153:	learn: 0.3927709	total: 189ms	remaining: 1.04s
154:	learn: 0.3921969	total: 191ms	remaining: 1.04s
155:	learn: 0.3916899	total: 192ms	remaining: 1.04s
156:	learn: 0.3913241	total: 193ms	remaining: 1.03s
157:	learn: 0.3912134	total: 194ms	remaining: 1.03s
158:	learn: 0.3909359	total: 195ms	remaining: 1.03s
159:	learn: 0.3905386	total: 201ms	remaining: 1.05s
160:	learn: 0.3899548	total: 202ms	remaining: 1.05s
161:	learn: 0.3893669	total: 204ms	remaining: 1.05s
162:	learn: 0.3890419	total: 205ms	remaining: 1.05s
163:	learn: 

419:	learn: 0.3367479	total: 538ms	remaining: 743ms
420:	learn: 0.3366508	total: 540ms	remaining: 742ms
421:	learn: 0.3365773	total: 541ms	remaining: 741ms
422:	learn: 0.3363496	total: 544ms	remaining: 742ms
423:	learn: 0.3362996	total: 547ms	remaining: 743ms
424:	learn: 0.3362069	total: 549ms	remaining: 742ms
425:	learn: 0.3361139	total: 550ms	remaining: 741ms
426:	learn: 0.3360534	total: 551ms	remaining: 739ms
427:	learn: 0.3360444	total: 551ms	remaining: 737ms
428:	learn: 0.3358830	total: 553ms	remaining: 735ms
429:	learn: 0.3356785	total: 554ms	remaining: 734ms
430:	learn: 0.3356033	total: 554ms	remaining: 732ms
431:	learn: 0.3355096	total: 555ms	remaining: 730ms
432:	learn: 0.3354675	total: 556ms	remaining: 728ms
433:	learn: 0.3353703	total: 558ms	remaining: 728ms
434:	learn: 0.3352151	total: 560ms	remaining: 727ms
435:	learn: 0.3351855	total: 561ms	remaining: 726ms
436:	learn: 0.3350961	total: 563ms	remaining: 725ms
437:	learn: 0.3350148	total: 567ms	remaining: 728ms
438:	learn: 

688:	learn: 0.3064100	total: 898ms	remaining: 406ms
689:	learn: 0.3061355	total: 901ms	remaining: 405ms
690:	learn: 0.3059922	total: 902ms	remaining: 403ms
691:	learn: 0.3058907	total: 905ms	remaining: 403ms
692:	learn: 0.3057413	total: 908ms	remaining: 402ms
693:	learn: 0.3055918	total: 911ms	remaining: 402ms
694:	learn: 0.3055241	total: 912ms	remaining: 400ms
695:	learn: 0.3053892	total: 913ms	remaining: 399ms
696:	learn: 0.3052526	total: 915ms	remaining: 398ms
697:	learn: 0.3051402	total: 919ms	remaining: 397ms
698:	learn: 0.3050854	total: 920ms	remaining: 396ms
699:	learn: 0.3050565	total: 921ms	remaining: 395ms
700:	learn: 0.3050513	total: 922ms	remaining: 393ms
701:	learn: 0.3047114	total: 924ms	remaining: 392ms
702:	learn: 0.3046125	total: 925ms	remaining: 391ms
703:	learn: 0.3046075	total: 926ms	remaining: 389ms
704:	learn: 0.3045121	total: 927ms	remaining: 388ms
705:	learn: 0.3045054	total: 928ms	remaining: 387ms
706:	learn: 0.3043999	total: 929ms	remaining: 385ms
707:	learn: 

937:	learn: 0.2798437	total: 1.26s	remaining: 83.6ms
938:	learn: 0.2797256	total: 1.27s	remaining: 82.3ms
939:	learn: 0.2796841	total: 1.27s	remaining: 81ms
940:	learn: 0.2795324	total: 1.27s	remaining: 79.7ms
941:	learn: 0.2794449	total: 1.27s	remaining: 78.6ms
942:	learn: 0.2793745	total: 1.28s	remaining: 77.3ms
943:	learn: 0.2791922	total: 1.28s	remaining: 76.1ms
944:	learn: 0.2790699	total: 1.28s	remaining: 74.8ms
945:	learn: 0.2789990	total: 1.29s	remaining: 73.7ms
946:	learn: 0.2789246	total: 1.29s	remaining: 72.4ms
947:	learn: 0.2788649	total: 1.29s	remaining: 71ms
948:	learn: 0.2786810	total: 1.3s	remaining: 69.8ms
949:	learn: 0.2786119	total: 1.3s	remaining: 68.4ms
950:	learn: 0.2785724	total: 1.3s	remaining: 67.1ms
951:	learn: 0.2785104	total: 1.3s	remaining: 65.7ms
952:	learn: 0.2784114	total: 1.3s	remaining: 64.3ms
953:	learn: 0.2783767	total: 1.3s	remaining: 62.9ms
954:	learn: 0.2781970	total: 1.31s	remaining: 61.6ms
955:	learn: 0.2781397	total: 1.31s	remaining: 60.2ms
956

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,84.399551,82.214073,1.855013
2,CatBoost,83.389450,81.339277,18.298809
3,Skl HistGBM,82.491582,80.886300,28.864290
4,LightGBM,82.267116,80.483654,4.717958
5,XGBoost,81.481481,80.011504,4.853219
6,Random Forest,81.369248,80.030678,3.073629
7,AdaBoost,80.583614,79.448279,1.710252
8,Extra Trees,80.134680,78.753236,2.432981
9,Decision Tree,79.685746,78.333813,0.313331


In [36]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [23]:
best_model = tree_classifiers["Skl GBM"].fit(x,y)
# Fit best model with all data

# YOUR CODE HERE


# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [24]:
test_pred = best_model.predict(x_test)


In [25]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [26]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [27]:
sub.to_csv("sub_with_cabin2.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"